In [2]:
!git clone https://github.com/JiamingZhuo/DUALFormer.git

Cloning into 'DUALFormer'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 32 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 1.79 MiB | 7.82 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/kaggle/working/DUALFormer


In [ ]:
# Kiểm tra phiên bản PyTorch và CUDA đang chạy trên Kaggle
import torch
import subprocess
import itertools
import os

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")

# Cài đặt PyTorch Geometric và các thư viện phụ trợ (Scatter, Sparse)
!pip install -q torch-geometric
!pip install -q pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{torch.__version__}.html

PyTorch: 2.8.0+cu126
CUDA: 12.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:00a 0:00:01


In [3]:
# Di chuyển vào thư mục dự án
%cd DUALFormer

# Tạo thư mục chứa kết quả (để tránh lỗi khi code cố gắng ghi file csv)
!mkdir -p results

/kaggle/working/DUALFormer


In [ ]:
util_content = """
import random
import numpy as np
import torch
import os
from torch_geometric.datasets import Planetoid, WebKB, Actor, Amazon, WikiCS, WikipediaNetwork, Coauthor
import torch_geometric.transforms as T
from torch_geometric.utils import to_dense_adj, dense_to_sparse, to_torch_coo_tensor

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def DataLoader(name):
    name = name.lower()
    root_path = 'datasets/'
    if name in ['cora', 'citeseer', 'pubmed']:
        # Dùng split='public' thay vì 'random' để khớp với paper
        dataset = Planetoid(root_path, name, split='public', transform=T.NormalizeFeatures())
    elif name in ['computers', 'photo']:
        dataset = Amazon(root_path, name, T.NormalizeFeatures())
    elif name in ['cs', 'physics']:
        dataset = Coauthor(root_path, name, T.NormalizeFeatures())
    elif name in ['chameleon', 'squirrel']:
        preProcDs = WikipediaNetwork(root=root_path, name=name, geom_gcn_preprocess=True, transform=T.NormalizeFeatures())
        dataset = WikipediaNetwork(root=root_path, name=name, geom_gcn_preprocess=True, transform=T.NormalizeFeatures())
        data = dataset[0]
        data.edge_index = preProcDs[0].edge_index
        dataset.data = data
        return dataset
    elif name in ['film']:
        dataset = Actor(root=root_path+'/Actor', transform=T.NormalizeFeatures())
        dataset.name = name
    elif name in ['texas', 'cornell', 'wisconsin']:
        dataset = WebKB(root=root_path, name=name, transform=T.NormalizeFeatures())
    elif name in ['wikics']:
        dataset = WikiCS(root=root_path+'/WikiCS', transform=T.NormalizeFeatures())
    else:
        raise ValueError(f'dataset {name} not supported in dataloader')
    return dataset

def dataset_split(data, run_id):
    if data.name in ['wikics', 'computers', 'photo', 'physics']:
        return get_split(num_samples=data.num_nodes, train_ratio=0.1, test_ratio=0.8)
    elif data.name in ['cora', 'citeseer', 'pubmed']:
        return get_public_split(data)

def get_public_split(data):
    return {
        'train': torch.where(data.train_mask)[0],
        'valid': torch.where(data.val_mask)[0],
        'test': torch.where(data.test_mask)[0]
    }

def get_split(num_samples: int, train_ratio: float = 0.1, test_ratio: float = 0.1):
    train_size = int(num_samples * train_ratio)
    test_size = int(num_samples * test_ratio)
    indices = torch.randperm(num_samples)
    return {
        'train': indices[:train_size],
        'test': indices[train_size: test_size + train_size],
        'valid': indices[test_size + train_size:]
    }
"""

with open('util.py', 'w') as f:
    f.write(util_content)

In [22]:
!cat util.py


import random
import numpy as np
import torch
import os
from torch_geometric.datasets import Planetoid, WebKB, Actor, Amazon, WikiCS, WikipediaNetwork, Coauthor
import torch_geometric.transforms as T
from torch_geometric.utils import to_dense_adj, dense_to_sparse, to_torch_coo_tensor

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def DataLoader(name):
    name = name.lower()
    root_path = 'datasets/'
    if name in ['cora', 'citeseer', 'pubmed']:
        # Dùng split='public' thay vì 'random' để khớp với paper
        dataset = Planetoid(root_path, name, split='public', transform=T.NormalizeFeatures())
    elif name in ['computers', 'photo']:
        dataset = Amazon(root_path, name, T.NormalizeFeatures())
    elif name in ['cs', 'physics']:
        dataset = Coauthor(root_path

In [ ]:
# Chạy Grid Search để tìm bộ tham số tốt nhất trên dataset Cora
# 1. Các tham số cố định chính xác theo Table 5 trong paper cho dataset Cora
fixed_params = {
    "dataset": "cora",
    "num_sa": "3",
    "num_gnns": "2",
    "hidden": "256",
    "alpha": "0.1",
    "lr": "0.001",
    "lr_sa": "0.001",
    "epochs": "300",
    "patience": "50",
    "runs": "5"  # Tìm nhanh tổ hợp tốt nhất bằng cách chạy 5 lần
}

# 2. Dải tham số cần tìm kiếm (Grid Search) theo Section D.3
search_space = {
    "wd": [1e-3, 1e-4, 1e-5],
    "dropout": [0.1, 0.3, 0.5],
    "heads": [2, 4]
}

# Tạo danh sách các tổ hợp 
keys, values = zip(*search_space.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

best_acc = 0
best_config = ""

print(f"Bắt đầu Grid Search trên {len(combinations)} tổ hợp...")

for config in combinations:
    print(f"\n--- Đang thử: WD={config['wd']}, Dropout={config['dropout']}, Heads={config['heads']} ---")
    
    cmd = [
        "python", "train.py",
        "--dataset", fixed_params["dataset"],
        "--num_sa", fixed_params["num_sa"],
        "--num_gnns", fixed_params["num_gnns"],
        "--hidden", fixed_params["hidden"],
        "--alpha", fixed_params["alpha"],
        "--lr", fixed_params["lr"],
        "--lr_sa", fixed_params["lr_sa"],
        "--weight_decay", str(config["wd"]),
        "--weight_decay_sa", str(config["wd"]),
        "--dropout", str(config["dropout"]),
        "--dropout_sa", str(config["dropout"]),
        "--num_heads", str(config["heads"]),
        "--epochs", fixed_params["epochs"],
        "--runs", fixed_params["runs"],
        "--patience", fixed_params["patience"]
    ]
    
    # Chạy lệnh và lấy kết quả từ log
    process = subprocess.run(cmd, capture_output=True, text=True)
    output = process.stdout
    
    # Tìm dòng chứa kết quả ACC cuối cùng
    try:
        acc_line = [line for line in output.split('\n') if "ACC, mean" in line][-1]
        current_acc = float(acc_line.split(':')[1].split('%')[0].strip())
        print(f">> Kết quả trung bình: {current_acc}%")
        
        if current_acc > best_acc:
            best_acc = current_acc
            best_config = " ".join(cmd[2:]) # Lưu lại các tham số để có thể chạy lại
    except:
        print("Lỗi khi đọc kết quả từ log.")

print("\n" + "="*50)
print(f"KẾT QUẢ CAO NHẤT: {best_acc}%")
print(f"Lệnh chạy bộ tham số này:\npython train.py {best_config} --runs 10")
print("="*50)

Bắt đầu Grid Search trên 18 tổ hợp...

--- Đang thử: WD=0.001, Dropout=0.1, Heads=2 ---
>> Kết quả trung bình: 80.2%

--- Đang thử: WD=0.001, Dropout=0.1, Heads=4 ---
>> Kết quả trung bình: 80.8%

--- Đang thử: WD=0.001, Dropout=0.3, Heads=2 ---
>> Kết quả trung bình: 81.5%

--- Đang thử: WD=0.001, Dropout=0.3, Heads=4 ---
>> Kết quả trung bình: 82.2%

--- Đang thử: WD=0.001, Dropout=0.5, Heads=2 ---
>> Kết quả trung bình: 82.84%

--- Đang thử: WD=0.001, Dropout=0.5, Heads=4 ---
>> Kết quả trung bình: 82.24%

--- Đang thử: WD=0.0001, Dropout=0.1, Heads=2 ---
>> Kết quả trung bình: 80.4%

--- Đang thử: WD=0.0001, Dropout=0.1, Heads=4 ---
>> Kết quả trung bình: 81.06%

--- Đang thử: WD=0.0001, Dropout=0.3, Heads=2 ---
>> Kết quả trung bình: 81.3%

--- Đang thử: WD=0.0001, Dropout=0.3, Heads=4 ---
>> Kết quả trung bình: 81.96%

--- Đang thử: WD=0.0001, Dropout=0.5, Heads=2 ---
>> Kết quả trung bình: 82.26%

--- Đang thử: WD=0.0001, Dropout=0.5, Heads=4 ---
>> Kết quả trung bình: 83.4%

--

In [25]:
!python train.py --dataset cora --num_sa 3 --num_gnns 2 --hidden 256 --alpha 0.1 --lr 0.001 --lr_sa 0.001 --weight_decay 0.0001 --weight_decay_sa 0.0001 --dropout 0.5 --dropout_sa 0.5 --num_heads 4 --epochs 300 --runs 5 --patience 50 --runs 10

data: cora
(T):  42%|█▋  | 126/300 [00:06<00:09, 18.22it/s, Train Loss=0.245, Val Loss=0.8]
1-th run, micro-f1:83.50%, macor-f1:82.08%; acc:83.50%;
(T):  42%|▊ | 126/300 [00:06<00:08, 19.59it/s, Train Loss=0.245, Val Loss=0.801]
2-th run, micro-f1:83.40%, macor-f1:81.99%; acc:83.40%;
(T):  42%|▊ | 126/300 [00:06<00:08, 19.44it/s, Train Loss=0.245, Val Loss=0.797]
3-th run, micro-f1:83.60%, macor-f1:82.15%; acc:83.60%;
(T):  42%|▊ | 126/300 [00:06<00:08, 19.38it/s, Train Loss=0.245, Val Loss=0.796]
4-th run, micro-f1:83.50%, macor-f1:82.09%; acc:83.50%;
(T):  42%|▊ | 126/300 [00:06<00:08, 19.36it/s, Train Loss=0.246, Val Loss=0.799]
5-th run, micro-f1:83.60%, macor-f1:82.26%; acc:83.60%;
(T):  42%|▊ | 126/300 [00:06<00:09, 19.17it/s, Train Loss=0.248, Val Loss=0.796]
6-th run, micro-f1:83.40%, macor-f1:82.01%; acc:83.40%;
(T):  42%|▊ | 126/300 [00:06<00:09, 19.07it/s, Train Loss=0.247, Val Loss=0.797]
7-th run, micro-f1:83.50%, macor-f1:82.09%; acc:83.50%;
(T):  42%|▊ | 126/300 [00:06<0

In [24]:
import pandas as pd
import glob

# Tìm file kết quả vừa sinh ra
result_files = glob.glob('results/*.csv')

if result_files:
    print("Tìm thấy các file kết quả:", result_files)
    # Đọc file mới nhất
    df = pd.read_csv(result_files[0], header=None)
    print(df)
else:
    print("Chưa thấy file kết quả nào.")

Tìm thấy các file kết quả: ['results/sgc_cora.csv']
                 0                1                2          3           4   \
0   mi:82.60 ± 0.00  ma:80.81 ± 0.00  ma:82.60 ± 0.00  seed:2024  epochs:200   
1   mi:80.52 ± 0.11  ma:79.20 ± 0.14  ma:80.52 ± 0.11  seed:2024  epochs:300   
2   mi:82.80 ± 0.00  ma:82.41 ± 0.00  ma:82.80 ± 0.00  seed:2024  epochs:200   
3   mi:79.96 ± 0.05  ma:78.31 ± 0.03  ma:79.96 ± 0.05  seed:2024  epochs:200   
4   mi:79.90 ± 0.00  ma:80.06 ± 0.00  ma:79.90 ± 0.00  seed:2024  epochs:200   
5   mi:79.10 ± 0.00  ma:77.13 ± 0.00  ma:79.10 ± 0.00  seed:2024  epochs:200   
6   mi:82.72 ± 0.09  ma:81.54 ± 0.08  ma:82.72 ± 0.09  seed:2024  epochs:300   
7   mi:82.33 ± 0.15  ma:81.31 ± 0.13  ma:82.33 ± 0.15  seed:2024  epochs:300   
8   mi:82.34 ± 0.14  ma:81.23 ± 0.10  ma:82.34 ± 0.14  seed:2024  epochs:300   
9   mi:80.20 ± 0.00  ma:79.50 ± 0.00  ma:80.20 ± 0.00  seed:2024  epochs:300   
10  mi:80.80 ± 0.00  ma:80.37 ± 0.00  ma:80.80 ± 0.00  seed:2024  ep